In [2]:
import cv2
import numpy as np
import os
from datetime import datetime

In [3]:
# Create a filename based on scale and threshold
output_folder = 'C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/27thNov'
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist

In [5]:
# Load the main image and the template
img = cv2.imread('data/sample.jpg')
img_gray = cv2.imread('data/sample.jpg', cv2.IMREAD_GRAYSCALE)
template = cv2.imread('data/test1.jpg', cv2.IMREAD_GRAYSCALE)

print(f'image shape: {img_gray.shape}')
print(f'template shape: {template.shape}')

# Store width and height of template in w and h 
w, h = template.shape[::-1] 

# Define scale and threshold
scale = 0.9
threshold = 0.799

# Prepare a list to hold all rotated templates
templates = []

# Resize and rotate the template
for i in range(4):
    if i == 0:
        # Original template
        template_resized = cv2.resize(template, (int(w * scale), int(h * scale)))
    else:
        # Rotate 90 degrees
        template_resized = cv2.rotate(template_resized, cv2.ROTATE_90_CLOCKWISE)
    
    templates.append(template_resized)

# Create an image to draw the detections
img_detected = img.copy()  # Create a copy of the original image to draw on
count_detected = 0
detected_points = set()  # To track unique detections

# Perform template matching for each template
for template_resized in templates:
    w_resized, h_resized = template_resized.shape[::-1]
    
    # Detect edges in the grayscale image and apply template matching
    result = cv2.matchTemplate(img_gray, template_resized, cv2.TM_CCOEFF_NORMED)

    # Set threshold for detection
    loc = np.where(result >= threshold)

    # Prepare bounding boxes for detected locations
    for pt in zip(*loc[::-1]):  # Switch columns and rows
        box = [pt[0], pt[1], pt[0] + w_resized, pt[1] + h_resized]  # [x1, y1, x2, y2]
        
        # Check if this box overlaps with previously detected boxes
        box_tuple = (box[0], box[1], box[2], box[3])
        if not any((box[0] < x2 and box[2] > x1 and box[1] < y2 and box[3] > y1) for (x1, y1, x2, y2) in detected_points):
            cv2.rectangle(img_detected, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)  # Red color
            detected_points.add(box_tuple)  # Add this box to the set of detected points
            count_detected += 1  # Increment the count for each unique detection


filename = f"detected_scale_{scale}_threshold_{threshold}_{current_time}.jpg"
filepath = os.path.join(output_folder, filename)

# Save the detected image
cv2.imwrite(filepath, img_detected)

# Print the count of detected items
print(f'File Stored at: {filepath}\nfile_name: {filename}')
print(f"Scale: {scale}, Threshold: {threshold}, Count Detected: {count_detected}")

# Optionally display the last detected image
# cv2.imshow("Detected", img_detected) 
# cv2.waitKey(0)
# cv2.destroyAllWindows()

image shape: (9000, 12600)
template shape: (64, 56)
File Stored at: C:/Users/pis05408.PINNACLE/Desktop/Suraj/ObjectDetectionFlorPlan/27thNov\detected_scale_0.9_threshold_0.799_20241127_091807.jpg
file_name: detected_scale_0.9_threshold_0.799_20241127_091807.jpg
Scale: 0.9, Threshold: 0.799, Count Detected: 0
